In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests9 as vuong_tests_fast

In [2]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params+1]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2


In [3]:

def gen_data(nobs=1000, a=0.25, num_params=4):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=15)
print(grad1.shape,hess1.shape)

(1000, 2) (2, 2)


In [4]:
num_sims = 500
trials =1000
adapt_c = True


# power 0.... need to see...

In [5]:
a1,a2 = np.sqrt(1.09-1), 0.00
num_params= 9


def gen_data2(nobs=1000, a1=np.sqrt(1.09-1), a2=0.00 , num_params=19):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a1*x[:,0] + a2/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

In [6]:

data_tuned_c = .9
calc_c =lambda nobs: (nobs**(-1/2), nobs**(-3/5)/350)

nobs = 250
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=True, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(c1,c2)
print(mc_out)

Quantiles: 1th: 0.81, 5th: 1.11, 10th: 1.30, 30th: 1.75, 50th: 2.08, 70th: 2.45, 90th: 3.10, 95th: 3.45, 99th: 4.21
bonus term 0.0412378035183607 // variance stats 5.064568500106632 //part1 0.2500894842310367 //part2 6.064568500106631 5.064568500106632
-1.6505886686909212 1.2126684110980224 1.8199623218161034
-1.7330642757276427 1.2126684110980224 1.737486714779382
---
Quantiles: 1th: 0.73, 5th: 0.98, 10th: 1.14, 30th: 1.57, 50th: 1.89, 70th: 2.22, 90th: 2.77, 95th: 3.06, 99th: 3.65
bonus term 0.03832716170562586 // variance stats 4.202793432852358 //part1 0.19940830522190064 //part2 5.202793432852357 4.202793432852358
-1.646355575439859 2.3770595353937174 1.62372309100215
-1.7230098988511107 2.3770595353937174 1.5470687675908985
---
Quantiles: 1th: 0.83, 5th: 1.13, 10th: 1.34, 30th: 1.78, 50th: 2.13, 70th: 2.53, 90th: 3.17, 95th: 3.50, 99th: 4.20
bonus term 0.03856718226331173 // variance stats 5.783563469094883 //part1 0.2616229287073256 //part2 6.783563469094884 5.783563469094883
-1

In [7]:

data_tuned_c = .9
calc_c =lambda nobs: (nobs**(-1/2), nobs**(-5/9)/350)

nobs = 250
c1,c2 = calc_c(nobs)

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=True, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(c1,c2)
print(mc_out)

Quantiles: 1th: 0.75, 5th: 1.02, 10th: 1.20, 30th: 1.59, 50th: 1.90, 70th: 2.24, 90th: 2.79, 95th: 3.08, 99th: 3.64
bonus term 0.05027185986138293 // variance stats 4.157846466776519 //part1 0.25929453476431824 //part2 5.157846466776519 4.157846466776519
-1.5819451106492806 2.3332544539428666 1.6604847890325656
-1.6824888303720464 2.3332544539428666 1.5599410693097997
---
Quantiles: 1th: 0.75, 5th: 1.03, 10th: 1.21, 30th: 1.62, 50th: 1.96, 70th: 2.33, 90th: 2.89, 95th: 3.20, 99th: 3.79
bonus term 0.05038092726389688 // variance stats 4.499374075388731 //part1 0.27706356528911985 //part2 5.499374075388732 4.499374075388731
-1.534066398881486 2.3339458340829755 1.6527752381338832
-1.6348282534092797 2.3339458340829755 1.5520133836060894
---
Quantiles: 1th: 0.79, 5th: 1.11, 10th: 1.30, 30th: 1.74, 50th: 2.07, 70th: 2.43, 90th: 3.00, 95th: 3.27, 99th: 3.83
bonus term 0.040882210580839684 // variance stats 6.3229343610869755 //part1 0.29937774461962446 //part2 7.3229343610869755 6.322934361

In [8]:

data_tuned_c = .8
calc_c =lambda nobs: (nobs**(-1/2), nobs**(-4/7)/350)

nobs = 250
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=True, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(c1,c2)
print(mc_out)

Quantiles: 1th: 0.81, 5th: 1.15, 10th: 1.32, 30th: 1.79, 50th: 2.16, 70th: 2.56, 90th: 3.21, 95th: 3.58, 99th: 4.20
bonus term 0.046206617781817574 // variance stats 4.307345227978123 //part1 0.24523447238533858 //part2 5.307345227978122 4.307345227978123
-1.6107553182600278 1.5353817582716989 1.6650523303426108
-1.7031685538236632 1.5353817582716989 1.5726390947789755
---
Quantiles: 1th: 0.77, 5th: 1.10, 10th: 1.28, 30th: 1.72, 50th: 2.07, 70th: 2.44, 90th: 3.04, 95th: 3.35, 99th: 3.97
bonus term 0.04489550892962185 // variance stats 3.9082384982556464 //part1 0.22035786532715013 //part2 4.908238498255646 3.9082384982556464
-1.4597174309441228 1.7877695511016896 1.6957979652339408
-1.5495084488033666 1.7877695511016896 1.606006947374697
---
Quantiles: 1th: 0.87, 5th: 1.20, 10th: 1.41, 30th: 1.85, 50th: 2.22, 70th: 2.62, 90th: 3.26, 95th: 3.58, 99th: 4.35
bonus term 0.039141714950570394 // variance stats 5.4475879991933915 //part1 0.2523696515831462 //part2 6.447587999193391 5.44758799

In [9]:
data_tuned_c = .95
calc_c =lambda nobs: (nobs**(-1/2), nobs**(-4/7)/350)

nobs = 250
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=True, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(c1,c2)
print(mc_out)

Quantiles: 1th: 0.81, 5th: 1.09, 10th: 1.28, 30th: 1.71, 50th: 2.05, 70th: 2.41, 90th: 3.01, 95th: 3.27, 99th: 3.81
bonus term 0.055616146869566785 // variance stats 4.845868494600521 //part1 0.32512468077587586 //part2 5.845868494600522 4.845868494600521
-1.5734315972233268 1.0334327480105654 1.642963565066662
-1.6846638909624605 1.0334327480105654 1.5317312713275284
---
Quantiles: 1th: 0.86, 5th: 1.18, 10th: 1.35, 30th: 1.79, 50th: 2.14, 70th: 2.51, 90th: 3.13, 95th: 3.42, 99th: 4.12
bonus term 0.04843085711683449 // variance stats 6.358848394621937 //part1 0.3563953351443819 //part2 7.358848394621936 6.358848394621937
-1.6008186562460422 3.207186401698821 1.9472006447553232
-1.6976803704797114 3.207186401698821 1.850338930521654
---
Quantiles: 1th: 0.86, 5th: 1.16, 10th: 1.35, 30th: 1.81, 50th: 2.17, 70th: 2.57, 90th: 3.23, 95th: 3.56, 99th: 4.20
bonus term 0.06164618402594543 // variance stats 5.266595801938762 //part1 0.38631171802253406 //part2 6.266595801938763 5.266595801938762

In [10]:

data_tuned_c = .9
calc_c =lambda nobs: (nobs**(-1/2), nobs**(-4/7)/350)

nobs = 250
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=True, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(c1,c2)
print(mc_out)

Quantiles: 1th: 0.90, 5th: 1.18, 10th: 1.37, 30th: 1.87, 50th: 2.22, 70th: 2.61, 90th: 3.24, 95th: 3.57, 99th: 4.18
bonus term 0.0531978510132801 // variance stats 5.0232980816087105 //part1 0.320426513953996 //part2 6.0232980816087105 5.0232980816087105
-1.5405029643710748 1.5465681143611951 1.6592875438642094
-1.6468986663976348 1.5465681143611951 1.5528918418376494
---
Quantiles: 1th: 0.83, 5th: 1.15, 10th: 1.34, 30th: 1.81, 50th: 2.18, 70th: 2.59, 90th: 3.31, 95th: 3.69, 99th: 4.41
bonus term 0.0598439216618045 // variance stats 4.580936877130429 //part1 0.3339851492744692 //part2 5.580936877130428 4.580936877130429
-1.5977960206074546 0.5015666225072363 1.6043507792800145
-1.7174838639310637 0.5015666225072363 1.4846629359564054
---
Quantiles: 1th: 0.91, 5th: 1.19, 10th: 1.40, 30th: 1.86, 50th: 2.21, 70th: 2.60, 90th: 3.19, 95th: 3.52, 99th: 4.11
bonus term 0.038997785388817645 // variance stats 6.954265555110702 //part1 0.3101987410438716 //part2 7.954265555110702 6.9542655551107

# a  = .25, k= 9

In [11]:
a = 0.25
num_params=9



In [12]:
nobs = 250
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=True, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

Quantiles: 1th: 0.84, 5th: 1.15, 10th: 1.34, 30th: 1.78, 50th: 2.15, 70th: 2.54, 90th: 3.15, 95th: 3.44, 99th: 4.09
bonus term 0.04698076924740617 // variance stats 5.426599508786602 //part1 0.3019265885677972 //part2 6.426599508786603 5.426599508786602
-1.5567966837660123 0.7443614916951407 1.5732196450009364
-1.6507582222608248 0.7443614916951407 1.4792581065061239
---
Quantiles: 1th: 0.88, 5th: 1.20, 10th: 1.38, 30th: 1.84, 50th: 2.20, 70th: 2.60, 90th: 3.17, 95th: 3.47, 99th: 4.08
bonus term 0.045621822602116995 // variance stats 5.72626855035577 //part1 0.30686463057852964 //part2 6.72626855035577 5.72626855035577
-1.6562896439212849 0.25280831931731723 1.6421114651161233
-1.7475332891255189 0.25280831931731723 1.5508678199118893
---
Quantiles: 1th: 0.88, 5th: 1.12, 10th: 1.31, 30th: 1.75, 50th: 2.10, 70th: 2.48, 90th: 3.08, 95th: 3.40, 99th: 3.99
bonus term 0.05760688181961723 // variance stats 3.9985793571228294 //part1 0.28795257029175314 //part2 4.998579357122829 3.99857935712

In [13]:
nobs = 500
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

Quantiles: 1th: 0.86, 5th: 1.18, 10th: 1.37, 30th: 1.81, 50th: 2.16, 70th: 2.54, 90th: 3.15, 95th: 3.44, 99th: 3.99
bonus term 0.005099175270637451 // variance stats 7.0377306045961845 //part1 0.040985797131002664 //part2 8.037730604596184 7.0377306045961845
-1.7450989373393908 1.5154779615944338 1.692998613540401
-1.7552972878806659 1.5154779615944338 1.6828002629991259
---
Quantiles: 1th: 0.81, 5th: 1.12, 10th: 1.30, 30th: 1.77, 50th: 2.13, 70th: 2.52, 90th: 3.15, 95th: 3.44, 99th: 4.19
bonus term 0.005644719708298885 // variance stats 6.260909176897698 //part1 0.040985797131002664 //part2 7.260909176897698 6.260909176897698
-1.7473431417720175 0.4421073931962846 1.7890472572025975
-1.7586325811886154 0.4421073931962846 1.7777578177859996
---
Quantiles: 1th: 0.80, 5th: 1.12, 10th: 1.30, 30th: 1.73, 50th: 2.07, 70th: 2.45, 90th: 3.05, 95th: 3.36, 99th: 3.95
bonus term 0.004937018902575564 // variance stats 7.301729837335854 //part1 0.040985797131002664 //part2 8.301729837335854 7.3017

In [14]:
nobs = 100
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=True, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

Quantiles: 1th: 0.86, 5th: 1.20, 10th: 1.37, 30th: 1.83, 50th: 2.19, 70th: 2.61, 90th: 3.17, 95th: 3.52, 99th: 4.16
bonus term 0.041702998028667164 // variance stats 3.9421713986643065 //part1 0.20610336409583285 //part2 4.942171398664307 3.9421713986643065
-1.360304178805551 1.652027347796621 1.5214856244729182
-1.4437101748628853 1.652027347796621 1.438079628415584
---
Quantiles: 1th: 0.75, 5th: 1.04, 10th: 1.24, 30th: 1.69, 50th: 2.06, 70th: 2.46, 90th: 3.06, 95th: 3.40, 99th: 4.04
bonus term 0.05481829500579255 // variance stats 2.5127485623917134 //part1 0.1925628869743626 //part2 3.512748562391713 2.5127485623917134
-1.4167775032088892 0.10169282869861587 1.4242703541587367
-1.5264140932204742 0.10169282869861587 1.3146337641471517
---
Quantiles: 1th: 0.77, 5th: 1.03, 10th: 1.21, 30th: 1.66, 50th: 2.02, 70th: 2.42, 90th: 3.06, 95th: 3.45, 99th: 4.21
bonus term 0.052648657754591696 // variance stats 2.659427507933854 //part1 0.1926639464429479 //part2 3.6594275079338545 2.65942750

# a  = .25, k= 4, something not right?

In [15]:
a = 0.25
num_params=4



In [16]:

nobs = 250
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.95 & 0.95 & 0.95 & 0.97 & 0.97 & 0.99   \\
Model 1 & 0.01 & 0.01 & 0.01 & 0.02 & 0.02 & 0.00   \\
Model 2 & 0.04 & 0.04 & 0.03 & 0.02 & 0.01 & 0.00   \\
\hline
\end{tabular}
(array([0.948, 0.012, 0.04 ]), array([0.948, 0.012, 0.04 ]), array([0.952, 0.014, 0.034]), array([0.966, 0.018, 0.016]), array([0.968, 0.018, 0.014]), array([0.992, 0.004, 0.004]), -1.610231282311871, 5.749146389608471, 5.739326094018919)


# a  = .25, K = 19

In [17]:
a = 0.25
num_params=19


In [18]:

nobs = 250
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.90 & 0.90 & 0.91 & 0.83 & 0.84 & 0.96   \\
Model 1 & 0.10 & 0.10 & 0.09 & 0.17 & 0.16 & 0.04   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.902, 0.098, 0.   ]), array([0.904, 0.096, 0.   ]), array([0.912, 0.088, 0.   ]), array([0.826, 0.174, 0.   ]), array([0.842, 0.158, 0.   ]), array([0.958, 0.042, 0.   ]), 4.293585956397237, 4.378464923568011, 4.552083234064904)


# evidence of power

In [19]:

nobs = 500
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.56 & 0.56 & 0.59 & 0.41 & 0.44 & 0.72   \\
Model 1 & 0.44 & 0.44 & 0.41 & 0.59 & 0.56 & 0.28   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.562, 0.438, 0.   ]), array([0.562, 0.438, 0.   ]), array([0.588, 0.412, 0.   ]), array([0.414, 0.586, 0.   ]), array([0.438, 0.562, 0.   ]), array([0.718, 0.282, 0.   ]), 9.436460724203176, 4.743398451779171, 5.013024152889418)


In [20]:

nobs = 100
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.49 & 0.50 & 0.58 & 0.41 & 0.40 & 0.67   \\
Model 1 & 0.51 & 0.50 & 0.42 & 0.59 & 0.60 & 0.33   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.486, 0.514, 0.   ]), array([0.498, 0.502, 0.   ]), array([0.584, 0.416, 0.   ]), array([0.414, 0.586, 0.   ]), array([0.398, 0.602, 0.   ]), array([0.672, 0.328, 0.   ]), 9.646050973265515, 4.691384372112556, 4.7444629620956515)


In [21]:

nobs = 250
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=4)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.59 & 0.59 & 0.62 & 0.46 & 0.48 & 0.75   \\
Model 1 & 0.41 & 0.41 & 0.38 & 0.54 & 0.52 & 0.25   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.592, 0.408, 0.   ]), array([0.592, 0.408, 0.   ]), array([0.62, 0.38, 0.  ]), array([0.46, 0.54, 0.  ]), array([0.476, 0.524, 0.   ]), array([0.748, 0.252, 0.   ]), 8.942627339406357, 4.653423241470801, 4.8783591059056475)


In [22]:

nobs = 250
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.55 & 0.55 & 0.60 & 0.42 & 0.43 & 0.71   \\
Model 1 & 0.45 & 0.45 & 0.40 & 0.58 & 0.57 & 0.29   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.554, 0.446, 0.   ]), array([0.554, 0.446, 0.   ]), array([0.596, 0.404, 0.   ]), array([0.418, 0.582, 0.   ]), array([0.43, 0.57, 0.  ]), array([0.714, 0.286, 0.   ]), 9.322239602355387, 4.865780001324123, 4.88052212475693)


# evidence of power 2

In [23]:
nobs = 250
c1,c2 = calc_c(nobs)



a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.92 & 0.93 & 0.94 & 0.98 & 0.98 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.08 & 0.07 & 0.06 & 0.02 & 0.02 & 0.00   \\
\hline
\end{tabular}
(array([0.922, 0.   , 0.078]), array([0.926, 0.   , 0.074]), array([0.944, 0.   , 0.056]), array([0.984, 0.   , 0.016]), array([0.984, 0.   , 0.016]), array([1., 0., 0.]), -3.6373544038131396, 2.632418603648592, 2.8999424897817354)


In [24]:
nobs = 500
c1,c2 = calc_c(nobs)



a2,a1 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.93 & 0.93 & 0.95 & 0.98 & 0.99 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.07 & 0.07 & 0.05 & 0.02 & 0.01 & 0.00   \\
\hline
\end{tabular}
(array([0.932, 0.   , 0.068]), array([0.934, 0.   , 0.066]), array([0.946, 0.   , 0.054]), array([0.984, 0.   , 0.016]), array([0.988, 0.   , 0.012]), array([1., 0., 0.]), -3.7364628229220687, 2.6554447532870555, 2.9668040049483513)


In [25]:
nobs = 100
c1,c2 = calc_c(nobs)



a2,a1 =np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.89 & 0.92 & 0.92 & 0.97 & 0.96 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.11 & 0.08 & 0.08 & 0.03 & 0.04 & 0.00   \\
\hline
\end{tabular}
(array([0.886, 0.   , 0.114]), array([0.92, 0.  , 0.08]), array([0.924, 0.   , 0.076]), array([0.968, 0.002, 0.03 ]), array([0.964, 0.   , 0.036]), array([0.998, 0.   , 0.002]), -3.8754986113289864, 2.7102229142295453, 2.885878246009428)


In [26]:
nobs = 250
c1,c2 = calc_c(nobs)


a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=4)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.22 & 0.22 & 0.27 & 0.47 & 0.50 & 0.76   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.78 & 0.78 & 0.73 & 0.53 & 0.50 & 0.24   \\
\hline
\end{tabular}
(array([0.216, 0.   , 0.784]), array([0.224, 0.   , 0.776]), array([0.27, 0.  , 0.73]), array([0.47, 0.  , 0.53]), array([0.5, 0. , 0.5]), array([0.762, 0.   , 0.238]), -12.051948538091807, 4.8365722846690185, 4.887767026156709)


In [27]:
nobs = 250
c1,c2 = calc_c(nobs)



a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.90 & 0.90 & 0.90 & 0.98 & 0.98 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.10 & 0.10 & 0.10 & 0.02 & 0.01 & 0.00   \\
\hline
\end{tabular}
(array([0.896, 0.   , 0.104]), array([0.898, 0.   , 0.102]), array([0.904, 0.   , 0.096]), array([0.976, 0.002, 0.022]), array([0.984, 0.002, 0.014]), array([0.998, 0.   , 0.002]), -3.942864176568571, 2.712471408143041, 2.9791828539228096)
